In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler




def Scaling(df,columns):
    for col in columns:
        scaler= StandardScaler()
        df[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1,1))
    return df[col]


def replace_nan_with_zero(df,variable):
    df_variables=[variable]
    for i in df_variables:
        df[i].replace(0,np.nan,inplace=True)
        return df[i].isnull().sum()

def return_median(df,variable):   
    temp_df = df[df[variable].notnull()]
    temp_df = temp_df[[variable, 'status_group']].groupby(['status_group'])[[variable]].median().reset_index()
    temp_col = temp_df[variable]
    return temp_col

import pandas as pd
import numpy as np



def installer_cl(df):
    """keep top 10 of installer"""

    if df['installer']=="DWE":
        return 'dwe'
    elif df['installer']=="Government":
        return 'gov'
    elif df['installer']=="RWE":
        return "rwe"
    elif df['installer']=="Commu":
        return"commu"
    elif df['installer']=="DANIDA":
        return"danida"
    elif df['installer']=="KKKT":
        return "kkkt"
    elif df['installer']=="Hesawa":
        return "hesewa"
    elif df['installer']=="0":
        return "unknown"
    elif df['installer']=="TCRS":
        return"tcrs"
    elif df['installer']=="Central government":
        return "central gov" 
    elif df['installer']=="NaN":
        return "nan"
    else:
        return "others"


def funder_cl(df):
    """keep top 10 of installer"""

    if df['funder']=="Government Of Tanzania":
        return 'gov of Tanzania'
    elif df['funder']=="Danida":
        return 'danida'
    elif df["funder"]=="Hesawa":
        return "hesawa"
    elif df['funder']=="Rwssp":
        return"rwssp"
    elif df['funder']=="World Vision":
        return"world vision"
    elif df['funder']=="Unicef":
        return "unicef"
    elif df['funder']=="Hesawa":
        return "hesewa"
    elif df['funder']=="Tasaf":
        return "tasaf"
    elif df['funder']=="District Council":
        return"district council"
    elif df['funder']=="Kkkt":
        return "kkkt" 
    elif df['funder']=="NaN":
        return "nan"
    elif df["funder"]=="0":
        return "unknown"
    else:
        return "others"


def year_cl(df):
    """Construction year grouping"""

    if df["construction_year"]==0:
        return 'unknown'
    elif 1960 <= df["construction_year"]<=1969:
        return "60s"
    elif 1970 <= df["construction_year"]<=1979:
        return "70s"
    elif 1980 <= df["construction_year"]<=1989:
        return "80s"
    elif 1990 <= df["construction_year"]<=1999:
        return "90s"
    elif 2000 <= df["construction_year"]<=2009:
        return "00s"
    elif 2010 <= df["construction_year"]<=2019:
        return "10s"



def pop_cl(df):
    """Population zero or not zero"""

    if df["population"]==0:
        return "1"
    else:
        return "0"


def pay_cl(df):
    """payment never pay , pay or unknown"""

    if df["payment"]=="never pay":
        return "NeverPay"
    elif df["payment"]=="unknown":
        return "Unknown"
    else:
        return "Pay"



















# read csv
df = pd.read_csv("data/Training_set_values.csv")
label = pd.read_csv("data/training_set_labels.csv")

# merge two datasets
df = pd.merge(df,label, on = "id")

# making a new numeric label for classification model
le = preprocessing.LabelEncoder()
le.fit(df["status_group"])
df["label"] =le.transform(df["status_group"])


# Longitude : missing value filled with median
replace_nan_with_zero(df,"longitude")
return_median(df,"longitude")

# Save 
save_it_to_variable = return_median(df,"longitude")

# replace median values with zero value
df.loc[(df['status_group'] == "functional" ) & (df["longitude"].isnull()), "longitude"] = save_it_to_variable[0]
df.loc[(df['status_group'] == "functional needs repair" ) & (df["longitude"].isnull()), "longitude"] = save_it_to_variable[1]
df.loc[(df['status_group'] == "non functional" ) & (df["longitude"].isnull()), "longitude"] = save_it_to_variable[2]


# gps_height : missing value filled with median
replace_nan_with_zero(df,"gps_height")
return_median(df,"gps_height")

# Save 
save_it_to_variable_gps = return_median(df,"gps_height")

# replace median values with zero value
df.loc[(df['status_group'] == "functional" ) & (df["gps_height"].isnull()), "gps_height"] = save_it_to_variable_gps[0]
df.loc[(df['status_group'] == "functional needs repair" ) & (df["gps_height"].isnull()), "gps_height"] = save_it_to_variable_gps[1]
df.loc[(df['status_group'] == "non functional" ) & (df["gps_height"].isnull()), "gps_height"] = save_it_to_variable_gps[2]

# Keep top 10 of funder and installer
df['installer_group'] = df.apply(lambda df: installer_cl(df), axis=1)
df['funder_group'] = df.apply(lambda df: installer_cl(df), axis=1)


# Sorting decades
df['construction_year_new'] = df.apply(lambda df: year_cl(df), axis=1)

# new columns tells if population is  zero or not zero
df["population_zero"] = df.apply(lambda df:pop_cl(df),axis =1)

# new columns tells if payment status is paid, not paid or unknown
df["payment_status"] = df.apply(lambda df:pay_cl(df),axis =1)



# Feature scaling : standarisaton
# list of numeric variables 
columns_list = ["gps_height","population","longitude","latitude"]
Scaling(df,columns_list)

#Remove unnecessary columns 
to_drop = ["funder","installer","payment","payment_type","wpt_name","recorded_by","num_private","extraction_type",
           "extraction_type_group","scheme_management","scheme_name","management",
          "quality_group","quantity_group","source","public_meeting","lga","ward",
           "subvillage","region_code","district_code","date_recorded","id",'label',
           "waterpoint_type_group","permit","construction_year","amount_tsh"]
df.drop(to_drop,inplace= True,axis =1)


# transform Categotical variable to numeric variable
df =pd.get_dummies(df,columns=["basin","region","extraction_type_class","management_group","payment_status","water_quality","quantity","source_type","source_class",
                                            "waterpoint_type","installer_group","construction_year_new","funder_group","population_zero","status_group"
                                           ])


df = df[['gps_height','longitude','latitude','population','basin_Internal',
 'basin_Lake Nyasa','basin_Lake Rukwa', 'basin_Lake Tanganyika',
 'basin_Lake Victoria', 'basin_Pangani', 'basin_Rufiji',
 'basin_Ruvuma / Southern Coast', 'basin_Wami / Ruvu', 'region_Arusha',
 'region_Dar es Salaam', 'region_Dodoma', 'region_Iringa', 'region_Kagera',
 'region_Kigoma', 'region_Kilimanjaro', 'region_Lindi', 'region_Manyara',
 'region_Mara', 'region_Mbeya', 'region_Morogoro', 'region_Mtwara',
 'region_Mwanza', 'region_Pwani', 'region_Rukwa' ,'region_Ruvuma',
 'region_Shinyanga', 'region_Singida', 'region_Tabora', 'region_Tanga',
 'extraction_type_class_gravity', 'extraction_type_class_handpump',
 'extraction_type_class_motorpump', 'extraction_type_class_other',
 'extraction_type_class_rope pump' ,'extraction_type_class_submersible',
 'extraction_type_class_wind-powered', 'management_group_commercial',
 'management_group_other', 'management_group_parastatal',
 'management_group_unknown' ,'management_group_user-group',
 'payment_status_NeverPay', 'payment_status_Pay', 'payment_status_Unknown',
 'water_quality_coloured', 'water_quality_fluoride',
 'water_quality_fluoride abandoned', 'water_quality_milky',
 'water_quality_salty', 'water_quality_salty abandoned',
 'water_quality_soft', 'water_quality_unknown', 'quantity_dry',
 'quantity_enough', 'quantity_insufficient', 'quantity_seasonal',
 'quantity_unknown', 'source_type_borehole', 'source_type_dam',
 'source_type_other' ,'source_type_rainwater harvesting',
 'source_type_river/lake', 'source_type_shallow well', 'source_type_spring',
 'source_class_groundwater', 'source_class_surface', 'source_class_unknown',
 'waterpoint_type_cattle trough', 'waterpoint_type_communal standpipe',
 'waterpoint_type_communal standpipe multiple', 'waterpoint_type_dam',
 'waterpoint_type_hand pump', 'waterpoint_type_improved spring',
 'waterpoint_type_other' ,'installer_group_central gov',
 'installer_group_commu', 'installer_group_danida', 'installer_group_dwe',
 'installer_group_gov', 'installer_group_hesewa', 'installer_group_kkkt',
 'installer_group_others', 'installer_group_rwe', 'installer_group_tcrs',
 'installer_group_unknown' ,'construction_year_new_00s',
 'construction_year_new_10s', 'construction_year_new_60s',
 'construction_year_new_70s', 'construction_year_new_80s',
 'construction_year_new_90s', 'construction_year_new_unknown',
 'funder_group_central gov', 'funder_group_commu' ,'funder_group_danida',
 'funder_group_dwe', 'funder_group_gov', 'funder_group_hesewa',
 'funder_group_kkkt', 'funder_group_others', 'funder_group_rwe',
 'funder_group_tcrs', 'funder_group_unknown', 'population_zero_0',
 'population_zero_1','status_group_functional','status_group_functional needs repair','status_group_non functional']]





# MOMO
# do something with missing data ( gps, popluration, water amount)





In [2]:
# filling space in columns' name with under score

df.columns = [c.replace(' ', '_') for c in df.columns]

In [ ]:
# save df

df.to_csv("data/tanzania_cleaned_df.csv",index=False)

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


In [3]:
df = pd.read_csv("data/tanzania_cleaned_df.csv")
df.shape

(59400, 113)

In [4]:
df.head()

,gps_height,longitude,latitude,population,basin_Internal,basin_Lake_Nyasa,basin_Lake_Rukwa,basin_Lake_Tanganyika,basin_Lake_Victoria,basin_Pangani,...,funder_group_kkkt,funder_group_others,funder_group_rwe,funder_group_tcrs,funder_group_unknown,population_zero_0,population_zero_1,status_group_functional,status_group_functional_needs_repair,status_group_non_functional
0,0.653165,-0.079198,-1.408791,-0.150399,0,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
1,0.671010,-0.172375,1.207934,0.212290,0,0,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0
2,-0.742690,0.902910,0.639751,0.148660,0,0,0,0,0,1,...,0,1,0,0,0,1,0,1,0,0
3,-1.581393,1.302165,-1.849720,-0.258570,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,0.373597,-1.561466,1.317271,-0.381587,0,0,0,0,1,0,...,0,1,0,0,0,0,1,1,0,0


In [5]:
df.columns

Index(['gps_height', 'longitude', 'latitude', 'population', 'basin_Internal',
       'basin_Lake_Nyasa', 'basin_Lake_Rukwa', 'basin_Lake_Tanganyika',
       'basin_Lake_Victoria', 'basin_Pangani',
       ...
       'funder_group_kkkt', 'funder_group_others', 'funder_group_rwe',
       'funder_group_tcrs', 'funder_group_unknown', 'population_zero_0',
       'population_zero_1', 'status_group_functional',
       'status_group_functional_needs_repair', 'status_group_non_functional'],
      dtype='object', length=113)

In [6]:
# Feature selection 
X = df.iloc[0:59400,0:110]
y= df[['status_group_functional','status_group_functional_needs_repair','status_group_non_functional']]


In [7]:
X

,gps_height,longitude,latitude,population,basin_Internal,basin_Lake_Nyasa,basin_Lake_Rukwa,basin_Lake_Tanganyika,basin_Lake_Victoria,basin_Pangani,...,funder_group_dwe,funder_group_gov,funder_group_hesewa,funder_group_kkkt,funder_group_others,funder_group_rwe,funder_group_tcrs,funder_group_unknown,population_zero_0,population_zero_1
0,0.653165,-0.079198,-1.408791,-0.150399,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.671010,-0.172375,1.207934,0.212290,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,-0.742690,0.902910,0.639751,0.148660,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3,-1.581393,1.302165,-1.849720,-0.258570,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.373597,-1.561466,1.317271,-0.381587,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,0.296270,0.789671,0.832379,-0.116463,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
59396,0.300236,0.042233,-1.142092,-0.262812,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
59397,0.373597,-0.437772,-1.033404,-0.381587,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
59398,0.373597,0.280239,-0.228290,-0.381587,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [8]:
np.count_nonzero(y[["status_group_functional_needs_repair"]]==1)

4317

In [9]:
# To use randomisesearch, y labels needs to be processed as one-hot-endoing style 
y

,status_group_functional,status_group_functional_needs_repair,status_group_non_functional
0,1,0,0
1,1,0,0
2,1,0,0
3,0,0,1
4,1,0,0
...,...,...,...
59395,1,0,0
59396,1,0,0
59397,1,0,0
59398,1,0,0


In [10]:
# To use randomisesearch, y labels needs to be processed as one-hot-endoing style 

y = preprocessing.label_binarize(y, classes=[0, 1, 2])
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [11]:
y.shape

(59400, 3)

In [12]:
print("the total df size " + str(df.shape))
print("y label 0 (functional) " + str(np.sum(y[:,0])))
print("y label 1 (need repair) " + str(np.sum(y[:,1])))
print("y label 2 (non functional) " + str(np.sum(y[:,2])))

the total df size (59400, 113)
y label 0 (functional) 32259
y label 1 (need repair) 4317
y label 2 (non functional) 22824


In [13]:
print("20% test before resampling " + str(11880))
print("80% train before resampling " + str(47520))

20% test before resampling 11880
80% train before resampling 47520


In [14]:
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN

Using TensorFlow backend.
C:\Users\yuka\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\yuka\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\yuka\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\yuka\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529

In [15]:
def split_data_resampling(X,y, test_percentage=0.2):
    #import pdb;pdb.set_trace()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_percentage, random_state=42)
    smote_enn = SMOTEENN(random_state=0)
    X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train)
    return X_train_resampled, y_train_resampled, X_test, y_test, X_train, y_train

In [16]:
X_train_resampled, y_train_resampled, X_test, y_test,X_train, y_train  = split_data_resampling(X,y)

In [17]:
print("Resampled train datasets")
print(len(X_train_resampled))
print(len(y_train_resampled))
print("The train datasets without resampling")
print(len(y_train))
print(len(y_train))

Resampled train datasets
58760
58760
The train datasets without resampling
47520
47520


In [18]:
print(len(X_test))
print(len(y_test))

11880
11880


In [19]:
print("Resampled y labels")
print(np.sum(y_train_resampled[:,0]))
print(np.sum(y_train_resampled[:,1]))
print(np.sum(y_train_resampled[:,2]))
print("The original y labels")
print(np.sum(y[:,0]))
print(np.sum(y[:,1]))
print(np.sum(y[:,2]))

Resampled y labels
17242
22137
19381
The original y labels
32259
4317
22824


# Base model (without hyperparameter turning) : XGB

In [20]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [21]:
from xgboost import XGBClassifier

In [22]:
# Need y data for base model (NO one hot encoding type)

In [23]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler




def Scaling(df,columns):
    for col in columns:
        scaler= StandardScaler()
        df[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1,1))
    return df[col]


def replace_nan_with_zero(df,variable):
    df_variables=[variable]
    for i in df_variables:
        df[i].replace(0,np.nan,inplace=True)
        return df[i].isnull().sum()

def return_median(df,variable):   
    temp_df = df[df[variable].notnull()]
    temp_df = temp_df[[variable, 'status_group']].groupby(['status_group'])[[variable]].median().reset_index()
    temp_col = temp_df[variable]
    return temp_col

import pandas as pd
import numpy as np



def installer_cl(df):
    """keep top 10 of installer"""

    if df['installer']=="DWE":
        return 'dwe'
    elif df['installer']=="Government":
        return 'gov'
    elif df['installer']=="RWE":
        return "rwe"
    elif df['installer']=="Commu":
        return"commu"
    elif df['installer']=="DANIDA":
        return"danida"
    elif df['installer']=="KKKT":
        return "kkkt"
    elif df['installer']=="Hesawa":
        return "hesewa"
    elif df['installer']=="0":
        return "unknown"
    elif df['installer']=="TCRS":
        return"tcrs"
    elif df['installer']=="Central government":
        return "central gov" 
    elif df['installer']=="NaN":
        return "nan"
    else:
        return "others"


def funder_cl(df):
    """keep top 10 of installer"""

    if df['funder']=="Government Of Tanzania":
        return 'gov of Tanzania'
    elif df['funder']=="Danida":
        return 'danida'
    elif df["funder"]=="Hesawa":
        return "hesawa"
    elif df['funder']=="Rwssp":
        return"rwssp"
    elif df['funder']=="World Vision":
        return"world vision"
    elif df['funder']=="Unicef":
        return "unicef"
    elif df['funder']=="Hesawa":
        return "hesewa"
    elif df['funder']=="Tasaf":
        return "tasaf"
    elif df['funder']=="District Council":
        return"district council"
    elif df['funder']=="Kkkt":
        return "kkkt" 
    elif df['funder']=="NaN":
        return "nan"
    elif df["funder"]=="0":
        return "unknown"
    else:
        return "others"


def year_cl(df):
    """Construction year grouping"""

    if df["construction_year"]==0:
        return 'unknown'
    elif 1960 <= df["construction_year"]<=1969:
        return "60s"
    elif 1970 <= df["construction_year"]<=1979:
        return "70s"
    elif 1980 <= df["construction_year"]<=1989:
        return "80s"
    elif 1990 <= df["construction_year"]<=1999:
        return "90s"
    elif 2000 <= df["construction_year"]<=2009:
        return "00s"
    elif 2010 <= df["construction_year"]<=2019:
        return "10s"



def pop_cl(df):
    """Population zero or not zero"""

    if df["population"]==0:
        return "1"
    else:
        return "0"


def pay_cl(df):
    """payment never pay , pay or unknown"""

    if df["payment"]=="never pay":
        return "NeverPay"
    elif df["payment"]=="unknown":
        return "Unknown"
    else:
        return "Pay"



















# read csv
df = pd.read_csv("data/Training_set_values.csv")
label = pd.read_csv("data/training_set_labels.csv")

# merge two datasets
df = pd.merge(df,label, on = "id")

# making a new numeric label for classification model
le = preprocessing.LabelEncoder()
le.fit(df["status_group"])
df["label"] =le.transform(df["status_group"])


# Longitude : missing value filled with median
replace_nan_with_zero(df,"longitude")
return_median(df,"longitude")

# Save 
save_it_to_variable = return_median(df,"longitude")

# replace median values with zero value
df.loc[(df['status_group'] == "functional" ) & (df["longitude"].isnull()), "longitude"] = save_it_to_variable[0]
df.loc[(df['status_group'] == "functional needs repair" ) & (df["longitude"].isnull()), "longitude"] = save_it_to_variable[1]
df.loc[(df['status_group'] == "non functional" ) & (df["longitude"].isnull()), "longitude"] = save_it_to_variable[2]


# gps_height : missing value filled with median
replace_nan_with_zero(df,"gps_height")
return_median(df,"gps_height")

# Save 
save_it_to_variable_gps = return_median(df,"gps_height")

# replace median values with zero value
df.loc[(df['status_group'] == "functional" ) & (df["gps_height"].isnull()), "gps_height"] = save_it_to_variable_gps[0]
df.loc[(df['status_group'] == "functional needs repair" ) & (df["gps_height"].isnull()), "gps_height"] = save_it_to_variable_gps[1]
df.loc[(df['status_group'] == "non functional" ) & (df["gps_height"].isnull()), "gps_height"] = save_it_to_variable_gps[2]

# Keep top 10 of funder and installer
df['installer_group'] = df.apply(lambda df: installer_cl(df), axis=1)
df['funder_group'] = df.apply(lambda df: installer_cl(df), axis=1)


# Sorting decades
df['construction_year_new'] = df.apply(lambda df: year_cl(df), axis=1)

# new columns tells if population is  zero or not zero
df["population_zero"] = df.apply(lambda df:pop_cl(df),axis =1)

# new columns tells if payment status is paid, not paid or unknown
df["payment_status"] = df.apply(lambda df:pay_cl(df),axis =1)



# Feature scaling : standarisaton
# list of numeric variables 
columns_list = ["gps_height","population","longitude","latitude"]
Scaling(df,columns_list)

#Remove unnecessary columns 
to_drop = ["funder","installer","payment","payment_type","wpt_name","recorded_by","num_private","extraction_type",
           "extraction_type_group","scheme_management","scheme_name","management",
          "quality_group","quantity_group","source","public_meeting","lga","ward",
           "subvillage","region_code","district_code","date_recorded","id",'label',
           "waterpoint_type_group","permit","construction_year","amount_tsh"]
df.drop(to_drop,inplace= True,axis =1)


# transform Categotical variable to numeric variable
df =pd.get_dummies(df,columns=["basin","region","extraction_type_class","management_group","payment_status","water_quality","quantity","source_type","source_class",
                                            "waterpoint_type","installer_group","construction_year_new","funder_group","population_zero"
                                           ])


df = df[['gps_height','longitude','latitude','population','basin_Internal',
 'basin_Lake Nyasa','basin_Lake Rukwa', 'basin_Lake Tanganyika',
 'basin_Lake Victoria', 'basin_Pangani', 'basin_Rufiji',
 'basin_Ruvuma / Southern Coast', 'basin_Wami / Ruvu', 'region_Arusha',
 'region_Dar es Salaam', 'region_Dodoma', 'region_Iringa', 'region_Kagera',
 'region_Kigoma', 'region_Kilimanjaro', 'region_Lindi', 'region_Manyara',
 'region_Mara', 'region_Mbeya', 'region_Morogoro', 'region_Mtwara',
 'region_Mwanza', 'region_Pwani', 'region_Rukwa' ,'region_Ruvuma',
 'region_Shinyanga', 'region_Singida', 'region_Tabora', 'region_Tanga',
 'extraction_type_class_gravity', 'extraction_type_class_handpump',
 'extraction_type_class_motorpump', 'extraction_type_class_other',
 'extraction_type_class_rope pump' ,'extraction_type_class_submersible',
 'extraction_type_class_wind-powered', 'management_group_commercial',
 'management_group_other', 'management_group_parastatal',
 'management_group_unknown' ,'management_group_user-group',
 'payment_status_NeverPay', 'payment_status_Pay', 'payment_status_Unknown',
 'water_quality_coloured', 'water_quality_fluoride',
 'water_quality_fluoride abandoned', 'water_quality_milky',
 'water_quality_salty', 'water_quality_salty abandoned',
 'water_quality_soft', 'water_quality_unknown', 'quantity_dry',
 'quantity_enough', 'quantity_insufficient', 'quantity_seasonal',
 'quantity_unknown', 'source_type_borehole', 'source_type_dam',
 'source_type_other' ,'source_type_rainwater harvesting',
 'source_type_river/lake', 'source_type_shallow well', 'source_type_spring',
 'source_class_groundwater', 'source_class_surface', 'source_class_unknown',
 'waterpoint_type_cattle trough', 'waterpoint_type_communal standpipe',
 'waterpoint_type_communal standpipe multiple', 'waterpoint_type_dam',
 'waterpoint_type_hand pump', 'waterpoint_type_improved spring',
 'waterpoint_type_other' ,'installer_group_central gov',
 'installer_group_commu', 'installer_group_danida', 'installer_group_dwe',
 'installer_group_gov', 'installer_group_hesewa', 'installer_group_kkkt',
 'installer_group_others', 'installer_group_rwe', 'installer_group_tcrs',
 'installer_group_unknown' ,'construction_year_new_00s',
 'construction_year_new_10s', 'construction_year_new_60s',
 'construction_year_new_70s', 'construction_year_new_80s',
 'construction_year_new_90s', 'construction_year_new_unknown',
 'funder_group_central gov', 'funder_group_commu' ,'funder_group_danida',
 'funder_group_dwe', 'funder_group_gov', 'funder_group_hesewa',
 'funder_group_kkkt', 'funder_group_others', 'funder_group_rwe',
 'funder_group_tcrs', 'funder_group_unknown', 'population_zero_0',
 'population_zero_1',"status_group"]]

# MOMO
# do something with missing data ( gps, popluration, water amount)


In [24]:
# filling space in columns' name with under score

df.columns = [c.replace(' ', '_') for c in df.columns]

In [25]:
df

,gps_height,longitude,latitude,population,basin_Internal,basin_Lake_Nyasa,basin_Lake_Rukwa,basin_Lake_Tanganyika,basin_Lake_Victoria,basin_Pangani,...,funder_group_gov,funder_group_hesewa,funder_group_kkkt,funder_group_others,funder_group_rwe,funder_group_tcrs,funder_group_unknown,population_zero_0,population_zero_1,status_group
0,0.653165,-0.079198,-1.408791,-0.150399,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,functional
1,0.671010,-0.172375,1.207934,0.212290,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,functional
2,-0.742690,0.902910,0.639751,0.148660,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,functional
3,-1.581393,1.302165,-1.849720,-0.258570,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,non functional
4,0.373597,-1.561466,1.317271,-0.381587,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,0.296270,0.789671,0.832379,-0.116463,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,functional
59396,0.300236,0.042233,-1.142092,-0.262812,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,functional
59397,0.373597,-0.437772,-1.033404,-0.381587,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,functional
59398,0.373597,0.280239,-0.228290,-0.381587,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,functional


In [26]:
# Feature selection 
X = df.iloc[0:59400,0:110]
y= df[['status_group']]

In [27]:
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

47520
47520
11880
11880


In [30]:
y_train

,status_group
3607,functional
50870,functional
20413,non functional
52806,non functional
50091,non functional
...,...
54343,functional
38158,functional
860,non functional
15795,functional


In [31]:
# change y_train data type for randomisesearch 

y_train_dum =pd.get_dummies(y_train)
y_train_dum

,status_group_functional,status_group_functional needs repair,status_group_non functional
3607,1,0,0
50870,1,0,0
20413,0,0,1
52806,0,0,1
50091,0,0,1
...,...,...,...
54343,1,0,0
38158,1,0,0
860,0,0,1
15795,1,0,0


In [32]:
y_train_dum = preprocessing.label_binarize(y_train_dum, classes=[0, 1, 2])
y_train_dum

array([[1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

In [33]:
len(y_train_dum)

47520

In [34]:
y_test

,status_group
2980,non functional
5246,functional
22659,functional
39888,non functional
13361,functional
...,...
17841,functional
20222,functional
40219,non functional
19599,functional


In [35]:
y_test_dum =pd.get_dummies(y_test)
y_test_dum

,status_group_functional,status_group_functional needs repair,status_group_non functional
2980,0,0,1
5246,1,0,0
22659,1,0,0
39888,0,0,1
13361,1,0,0
...,...,...,...
17841,1,0,0
20222,1,0,0
40219,0,0,1
19599,1,0,0


In [36]:
y_test_dum = preprocessing.label_binarize(y_test_dum, classes=[0, 1, 2])
y_test_dum

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

In [37]:
len(y_test_dum)

11880

In [38]:
def data_resampling(X_train,y_train_dum, test_percentage=0.2):
    smote_enn = SMOTEENN(random_state=0)
    X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train_dum)
    return X_train_resampled, y_train_resampled

In [39]:
X_train_resampled, y_train_resampled = data_resampling(X_train,y_train_dum)

# Resampling train and test for grid and randomised search

In [40]:
print("Resampled train datasets")
print(len(X_train_resampled))
print(len(y_train_resampled))
print("The train datasets without resampling")
print(len(X_train))
print(len(y_train))

Resampled train datasets
58760
58760
The train datasets without resampling
47520
47520


In [41]:
print(len(X_test))
print(len(y_test))

11880
11880


In [42]:
print("Resampled y labels")
print(np.sum(y_train_resampled[:,0]))
print(np.sum(y_train_resampled[:,1]))
print(np.sum(y_train_resampled[:,2]))
print("The original y labels")
print(np.sum(y_train_dum[:,0]))
print(np.sum(y_train_dum[:,1]))
print(np.sum(y_train_dum[:,2]))
print(y_train.status_group.value_counts())

Resampled y labels
17242
22137
19381
The original y labels
25802
3466
18252
functional                 25802
non functional             18252
functional needs repair     3466
Name: status_group, dtype: int64


# Resampling train and test for modeling

In [43]:
def data_resampling(X_train,y_train, test_percentage=0.2):
    smote_enn = SMOTEENN(random_state=0)
    X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train)
    return X_train_resampled, y_train_resampled

In [44]:
X_train_resampled2, y_train_resampled2 = data_resampling(X_train,y_train)

In [45]:
y_train_resampled2

,status_group
0,functional
1,functional
2,functional
3,functional
4,functional
...,...
58755,non functional
58756,non functional
58757,non functional
58758,non functional


In [46]:
print(len(X_train_resampled2))
print(len(y_train_resampled2))

58760
58760


In [47]:
print(y_train_resampled2.status_group.value_counts())

functional needs repair    22137
non functional             19381
functional                 17242
Name: status_group, dtype: int64


# from the link https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html

# Creates folds manually, and upsamples within each fold. Returns an array of validation (recall) scores

In [61]:
from xgboost import XGBClassifier
XGB =XGBClassifier()

In [63]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)

cross_val_score(XGB, X_train, y_train, cv=kf, scoring='f1_macro')

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0.81810463, 0.80938806, 0.80551056, 0.79654798, 0.81974877])

In [54]:
from sklearn.model_selection import KFold

In [55]:
kf = KFold(n_splits=5, random_state=42, shuffle=False)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [65]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
from sklearn.multiclass import OneVsRestClassifier

In [72]:
params = {'min_child_weight': [1],
 'max_depth': [5,10,15],
 'bootstrap': [True],
 'booster': ['gbtree']}

In [73]:
grid_no_up = GridSearchCV(XGB, param_grid=params, cv=kf, 
                          scoring='f1_macro').fit(X_train, y_train)

In [74]:
grid_no_up.best_score_

0.8132054831901684

In [75]:
grid_no_up.best_params_

{'booster': 'gbtree',
 'bootstrap': True,
 'max_depth': 10,
 'min_child_weight': 1}

In [80]:
from sklearn.metrics import f1_score

In [82]:
f1_score(y_test, grid_no_up.predict(X_test),average='micro')

0.8751515151515151

In [58]:
from imblearn.over_sampling import SMOTE

In [76]:
example_params = {'min_child_weight': [1],
 'max_depth': [5,10,15],
 'bootstrap': [True],
 'booster': ['gbtree']}

In [79]:
y

,status_group
0,functional
1,functional
2,functional
3,non functional
4,functional
...,...
59395,functional
59396,functional
59397,functional
59398,functional


In [86]:
smoter = SMOTE(random_state=42)

In [87]:
scores = []

In [121]:
for train_fold_index, val_fold_index in KFold(n_splits=5, random_state=42).split(X_train, y_train):
        # Get the training data
        X_train_fold, y_train_fold = X_train.iloc[train_fold_index], y_train.iloc[train_fold_index]
        # Get the validation data
        X_val_fold, y_val_fold = X_train.iloc[val_fold_index], y_train.iloc[val_fold_index]
        
         # Upsample only the data in the training section
        X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(X_train_fold,
                                                                           y_train_fold)
         # Fit the model on the upsampled training data
        model_obj = XGBClassifier(example_params).fit(X_train_fold_upsample, y_train_fold_upsample)
        # Score the model on the (non-upsampled) validation data
        score = f1_score(y_val_fold, model_obj.predict(X_val_fold),average='micro')
        scores.append(score)

IndentationError: unexpected indent (<ipython-input-121-d9dbb90fcc15>, line 15)

# Base model Resampled X and y train's cross validation F1 macro score 

# XGB base model

In [124]:
scores

[0.8521885521885522,
 0.8491582491582491,
 0.8509539842873176,
 0.8518518518518519,
 0.8560044893378227]

In [125]:
score.mean()

0.8560044893378227

In [126]:
X_train_fold_upsample

,gps_height,longitude,latitude,population,basin_Internal,basin_Lake_Nyasa,basin_Lake_Rukwa,basin_Lake_Tanganyika,basin_Lake_Victoria,basin_Pangani,...,funder_group_dwe,funder_group_gov,funder_group_hesewa,funder_group_kkkt,funder_group_others,funder_group_rwe,funder_group_tcrs,funder_group_unknown,population_zero_0,population_zero_1
0,-0.126055,-1.063734,0.223320,-0.381587,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.373597,-1.235910,0.740173,-0.381587,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1.057646,-0.105608,0.340415,-0.379466,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,1.632643,-0.308977,-1.329658,-0.381587,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,-0.354072,-0.006243,-0.691718,-0.379466,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57988,-0.570102,-1.838395,-0.361754,0.105761,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
57989,-0.024866,-0.033639,0.573368,-0.039051,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
57990,-1.938375,1.370116,-0.033788,0.577519,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
57991,-0.126055,-1.326832,1.475636,-0.381587,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1


In [127]:
y_train_fold_upsample

,status_group
0,non functional
1,functional
2,non functional
3,functional
4,functional
...,...
57988,non functional
57989,non functional
57990,non functional
57991,non functional


In [128]:
y_train_fold_upsample.status_group.value_counts()

functional needs repair    19331
non functional             19331
functional                 19331
Name: status_group, dtype: int64

In [139]:
example_params = {'booster': 'gbtree',
 'bootstrap': True,
 'max_depth': 10,
 'min_child_weight': 1}

In [152]:
example_params_rfc = {'max_depth': 10}

In [144]:

def score_model(model, params, cv=None):
    """
    Creates folds manually, and upsamples within each fold.
    Returns an array of validation (recall) scores
    """
    if cv is None:
        cv = KFold(n_splits=5, random_state=42)

    smoter = SMOTE(random_state=42)
    
    scores = []
    for train_fold_index, val_fold_index in KFold(n_splits=5, random_state=42).split(X_train, y_train):
        # Get the training data
        X_train_fold, y_train_fold = X_train.iloc[train_fold_index], y_train.iloc[train_fold_index]
        # Get the validation data
        X_val_fold, y_val_fold = X_train.iloc[val_fold_index], y_train.iloc[val_fold_index]
        
         # Upsample only the data in the training section
        X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(X_train_fold,
                                                                           y_train_fold)
         # Fit the model on the upsampled training data
        model_upsample = model(**params).fit(X_train_fold_upsample, y_train_fold_upsample)
        # Score the model on the (non-upsampled) validation data
        score = f1_score(y_val_fold, model_upsample.predict(X_val_fold),average='micro')
        scores.append(score)
        score_mean=score.mean()
    return np.array(scores),score_mean



In [145]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
GBC = GradientBoostingClassifier()
RFC =RandomForestClassifier()
svc =SVC()
XGB = XGBClassifier()

In [148]:
score_model(XGBClassifier, example_params, cv=kf)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shap

(array([0.8664422 , 0.86498316, 0.86004489, 0.86655443, 0.86835017]),
 0.8683501683501683)

In [150]:
score_model(GradientBoostingClassifier, cv=kf)

TypeError: score_model() missing 1 required positional argument: 'params'

In [153]:
score_model(RandomForestClassifier, example_params_rfc, cv=kf)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
C:\Users\yuka\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\yuka\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\yuka\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\yuka\Anaconda3\lib\site-packages\ipykerne

(array([0.81728395, 0.81391695, 0.81279461, 0.82356902, 0.82255892]),
 0.8225589225589225)

In [ ]:
score_model(svc, example_params, cv=kf)

# Base model with X y train without resampling

In [22]:
from xgboost import XGBClassifier

In [113]:
XGB =XGBClassifier()
XGB.fit(X_train,y_train)
y_pred =XGB.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                         precision    recall  f1-score   support

             functional       0.85      0.94      0.89      6457
functional needs repair       0.86      0.53      0.65       851
         non functional       0.90      0.83      0.87      4572

               accuracy                           0.87     11880
              macro avg       0.87      0.77      0.80     11880
           weighted avg       0.87      0.87      0.87     11880

0.8704545454545455


# Cross validation scores Base model with X y train without resampling

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [116]:
# cross val X_train, y_train datasets

cross_validation_score_base = cross_val_score(XGB, X_train, y_train)
print("Estimate the Accuracy of Base XGB classifier(cross validation score): %0.2f (+/- %0.2f)" % (cross_validation_score_base.mean(), cross_validation_score_base.std() * 2))

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Estimate the Accuracy of Base XGB classifier(cross validation score): 0.87 (+/- 0.01)


In [117]:
print(cross_validation_score_base)

[0.86752946 0.87636785 0.86374158 0.87268519 0.86731902]


# Function that returns train a model and return model's accuracy and F1_macro

In [24]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [25]:
def modeling(clf,X_train,y_train,X_test,y_test):
    """returns train a model and return model's accuracy and F1_macro"""
    
    clf.fit(X_train,y_train)
    y_pred =clf.predict(X_test)
    clf_report = classification_report(y_test,y_pred)
    clf_acc_score = accuracy_score(y_test, y_pred)
    f1score=f1_score(y_test, y_pred, average='macro')
    return  clf_report, clf_acc_score, f1score
    print(clf_report)

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [27]:
GBC = GradientBoostingClassifier()
GBC_clf_report, GBC_clf_acc_score, GBC_f1score = modeling(GBC,X_train,y_train,X_test,y_test)


RFC =RandomForestClassifier()
RFC_clf_report, RFC_clf_acc_score, RFC_f1score = modeling(RFC,X_train,y_train,X_test,y_test)

svc =SVC()
svc_clf_report, svc_clf_acc_score, svc_f1score = modeling(svc,X_train,y_train,X_test,y_test)


C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
XGB =XGBClassifier()
XGB_clf_report, XGB_clf_acc_score, XGB_f1score = modeling(XGB,X_train,y_train,X_test,y_test)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
results={'Classifier':["XGB Classifier","Gradient Boosting Classifier","Random Forest Classifier","SVC"],
'Accuracy':[XGB_clf_acc_score,GBC_clf_acc_score,RFC_clf_acc_score,svc_clf_acc_score],
'F1_macro':[XGB_f1score,GBC_f1score,RFC_f1score,svc_f1score]}

In [30]:
score_report_df =pd.DataFrame(data=results,columns=["Classifier","Accuracy","F1_macro"])
print(score_report_df)

                     Classifier  Accuracy  F1_macro
0                XGB Classifier  0.870455  0.804928
1  Gradient Boosting Classifier  0.847475  0.776302
2      Random Forest Classifier  0.876263  0.814413
3                           SVC  0.837710  0.696662


In [31]:
print("Base models Train_Test Split 80:20 validation")
score_report_df

Base models Train_Test Split 80:20 validation


,Classifier,Accuracy,F1_macro
0,XGB Classifier,0.870455,0.804928
1,Gradient Boosting Classifier,0.847475,0.776302
2,Random Forest Classifier,0.876263,0.814413
3,SVC,0.837710,0.696662


In [44]:
def return_x_val_accuracy(clf,X_train,y_train,kfold=5):
    """Return kfold cross validation scores of accuracy and its mean. kfold = 5 as default """
    cross_validation_score = cross_val_score(clf,X_train,y_train,cv=kfold)
    estimated_accuracy = (cross_validation_score.mean(), cross_validation_score.std() * 2)
    
    print("Estimate cross validation accuracy: %0.2f (+/- %0.2f)" % (cross_validation_score.mean(), cross_validation_score.std() * 2))
    print("Cross validation scores " + str(cross_validation_score))
    return cross_validation_score,estimated_accuracy


In [46]:
cross_validation_score_base_gbc,estimated_accuracy_base_gbc = return_x_val_accuracy(GBC,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Estimate cross validation accuracy: 0.85 (+/- 0.01)
Cross validation scores [0.84648569 0.85332492 0.8420665  0.84859007 0.84469697]


In [47]:
cross_validation_score_base_rfc,estimated_accuracy_base_rfc = return_x_val_accuracy(RFC,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was pa

Estimate cross validation accuracy: 0.87 (+/- 0.00)
Cross validation scores [0.87268519 0.87521044 0.86952862 0.87142256 0.87068603]


In [48]:
cross_validation_score_base_svc,estimated_accuracy_base_svc = return_x_val_accuracy(svc,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate cross validation accuracy: 0.84 (+/- 0.01)
Cross validation scores [0.83901515 0.84122475 0.83164983 0.83743687 0.83154461]


In [49]:
cross_validation_score_base_xgb,estimated_accuracy_base_xgb = return_x_val_accuracy(XGB,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Estimate cross validation accuracy: 0.87 (+/- 0.01)
Cross validation scores [0.86752946 0.87636785 0.86374158 0.87268519 0.86731902]


In [50]:
def return_x_val_f1_macro(clf,X_train,y_train,kfold=5):
    """Return kfold cross validation scores of f1_macro and its mean kfold = 5 as default """
    cross_validation_score_f1 = cross_val_score(clf,X_train,y_train,cv=kfold,scoring='f1_macro')
    estimated_f1 = (cross_validation_score_f1.mean(), cross_validation_score_f1.std() * 2)
    
    print("Estimate cross validation F1_macro: %0.2f (+/- %0.2f)" % (cross_validation_score_f1.mean(), cross_validation_score_f1.std() * 2))
    print("F1_macro Cross validation scores: " + str(cross_validation_score_f1))
    return cross_validation_score_f1,estimated_f1


In [52]:
cross_validation_score_f1_base_xgb,estimated_f1_base_xgb = return_x_val_f1_macro(XGB,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Estimate cross validation F1_macro: 0.81 (+/- 0.01)
F1_macro Cross validation scores: [0.80488188 0.81480204 0.79617152 0.81192296 0.80342481]


In [59]:
cross_validation_score_f1_base_xgb

array([0.80488188, 0.81480204, 0.79617152, 0.81192296, 0.80342481])

In [58]:
estimated_f1_base_xgb


(0.8062406411928424, 0.01316810135582849)

In [51]:
cross_validation_score_f1_base_gbc,estimated_f1_base_gbc = return_x_val_f1_macro(GBC,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Estimate cross validation F1_macro: 0.78 (+/- 0.02)
F1_macro Cross validation scores: [0.78297804 0.79311005 0.76543342 0.78836523 0.77935998]


In [53]:
cross_validation_score_f1_base_rfc,estimated_f1_base_rfc = return_x_val_f1_macro(RFC,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was pa

Estimate cross validation F1_macro: 0.81 (+/- 0.01)
F1_macro Cross validation scores: [0.80734946 0.81471298 0.79997518 0.81030385 0.81146923]


In [54]:
cross_validation_score_f1_base_svc,estimated_f1_base_svc = return_x_val_f1_macro(svc,X_train,y_train)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate cross validation F1_macro: 0.69 (+/- 0.02)
F1_macro Cross validation scores: [0.70103363 0.70145815 0.68161111 0.69209729 0.68405866]


In [55]:
results_x_val={'Classifier':["XGB Classifier","Gradient Boosting Classifier","Random Forest Classifier","SVC"],
'Cross validation accuracy':[estimated_accuracy_base_xgb,estimated_accuracy_base_gbc,estimated_accuracy_base_rfc,estimated_accuracy_base_svc],
'Cross validation F1_macro':[estimated_f1_base_xgb,estimated_f1_base_gbc,estimated_f1_base_rfc,estimated_f1_base_svc]}

In [56]:
score_report_df_x_val_base =pd.DataFrame(data=results_x_val,columns=['Classifier','Cross validation accuracy','Cross validation F1_macro'])

In [57]:
print("Base models 5folds cross validation (Train_Test Split 80:20)")
score_report_df_x_val_base

Base models 5folds cross validation (Train_Test Split 80:20)


,Classifier,Cross validation accuracy,Cross validation F1_macro
0,XGB Classifier,"(0.8695286195286196, 0.008904275785594567)","(0.8062406411928424, 0.01316810135582849)"
1,Gradient Boosting Classifier,"(0.8470328282828283, 0.0076123787519490884)","(0.7818493446057356, 0.01889367499062976)"
2,Random Forest Classifier,"(0.8719065656565655, 0.003888488034495354)","(0.8087621418773745, 0.009972924824283975)"
3,SVC,"(0.8361742424242424, 0.007852457152775046)","(0.6920517702741111, 0.01654214233144192)"


In [100]:
print("Base models Train_Test Split 80:20 validation")
score_report_df

Base models Train_Test Split 80:20 validation


,Classifier,Accuracy,F1_macro
0,XGB Classifier,0.870455,0.804928
1,Gradient Boosting Classifier,0.847475,0.776302
2,Random Forest Classifier,0.876263,0.814413
3,SVC,0.837710,0.696662


# Base mode, Resampling X_train and y_train

In [84]:
XGB =XGBClassifier()
XGB_clf_report_r, XGB_clf_acc_score_r, XGB_f1score_r = modeling(XGB,X_train_resampled2,y_train_resampled2,X_test,y_test)

In [86]:
print(XGB_clf_report_r)

                         precision    recall  f1-score   support

             functional       0.90      0.84      0.86      6457
functional needs repair       0.43      0.76      0.55       851
         non functional       0.88      0.84      0.86      4572

               accuracy                           0.83     11880
              macro avg       0.74      0.81      0.76     11880
           weighted avg       0.86      0.83      0.84     11880



In [87]:

GBC = GradientBoostingClassifier()
GBC_clf_report_r, GBC_clf_acc_score_r, GBC_f1score_r = modeling(GBC,X_train_resampled2,y_train_resampled2,X_test,y_test)


RFC =RandomForestClassifier()
RFC_clf_report_r, RFC_clf_acc_score_r, RFC_f1score_r = modeling(RFC,X_train_resampled2,y_train_resampled2,X_test,y_test)

svc =SVC()
svc_clf_report_r, svc_clf_acc_score_r, svc_f1score_r = modeling(svc,X_train_resampled2,y_train_resampled2,X_test,y_test)


C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [88]:
results_base_r={'Classifier':["XGB Classifier","Gradient Boosting Classifier","Random Forest Classifier","SVC"],
'Resampling Accuracy':[XGB_clf_acc_score_r,GBC_clf_acc_score_r,RFC_clf_acc_score_r,svc_clf_acc_score_r],
'Resampling F1_macro':[XGB_f1score_r,GBC_f1score_r,RFC_f1score_r,svc_f1score_r]}

In [89]:
score_report_df_base_r =pd.DataFrame(data=results_base_r,columns=["Classifier","Resampling Accuracy","Resampling F1_macro"])
print(score_report_df_base_r)

                     Classifier  Resampling Accuracy  Resampling F1_macro
0                XGB Classifier             0.831650             0.757909
1  Gradient Boosting Classifier             0.780808             0.703733
2      Random Forest Classifier             0.828199             0.753421
3                           SVC             0.810774             0.734234


In [90]:
print("Base models trained by resampled Train_Test Split 80:20 validation")
score_report_df_base_r

Base models trained by resampled Train_Test Split 80:20 validation


,Classifier,Resampling Accuracy,Resampling F1_macro
0,XGB Classifier,0.831650,0.757909
1,Gradient Boosting Classifier,0.780808,0.703733
2,Random Forest Classifier,0.828199,0.753421
3,SVC,0.810774,0.734234


In [91]:
score_report_df

,Classifier,Accuracy,F1_macro
0,XGB Classifier,0.870455,0.804928
1,Gradient Boosting Classifier,0.847475,0.776302
2,Random Forest Classifier,0.876263,0.814413
3,SVC,0.837710,0.696662


# base model Corss validation with resampling 

# accuracy

In [ ]:
def return_x_val_accuracy(clf,X_train,y_train,kfold=5):
    """Return kfold cross validation scores of accuracy and its mean. kfold = 5 as default """
    cross_validation_score = cross_val_score(clf,X_train,y_train,cv=kfold)
    estimated_accuracy = (cross_validation_score.mean(), cross_validation_score.std() * 2)
    
    print("Estimate cross validation accuracy: %0.2f (+/- %0.2f)" % (cross_validation_score.mean(), cross_validation_score.std() * 2))
    print("Cross validation scores " + str(cross_validation_score))
    return cross_validation_score,estimated_accuracy

In [92]:
cross_validation_score_base_xgb_r,estimated_accuracy_base_xgb_r = return_x_val_accuracy(XGB,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Estimate cross validation accuracy: 0.97 (+/- 0.01)
Cross validation scores [0.96307012 0.976855   0.9758339  0.97251532 0.96451668]


In [93]:
cross_validation_score_base_gbc_r,estimated_accuracy_base_gbc_r = return_x_val_accuracy(GBC,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Estimate cross validation accuracy: 0.91 (+/- 0.01)
Cross validation scores [0.90622873 0.90818584 0.9127808  0.90537781 0.8980599 ]


In [94]:
cross_validation_score_base_rfc_r,estimated_accuracy_base_rfc_r = return_x_val_accuracy(RFC,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was pa

Estimate cross validation accuracy: 0.99 (+/- 0.00)
Cross validation scores [0.98604493 0.98919333 0.99140572 0.98970388 0.98468346]


In [95]:
cross_validation_score_base_svc_r,estimated_accuracy_base_svc_r = return_x_val_accuracy(svc,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate cross validation accuracy: 0.94 (+/- 0.01)
Cross validation scores [0.93626617 0.9485194  0.94911504 0.94528591 0.93745745]


# f1_macro

In [ ]:
def return_x_val_f1_macro(clf,X_train,y_train,kfold=5):
    """Return kfold cross validation scores of f1_macro and its mean kfold = 5 as default """
    cross_validation_score_f1 = cross_val_score(clf,X_train,y_train,cv=kfold,scoring='f1_macro')
    estimated_f1 = (cross_validation_score_f1.mean(), cross_validation_score_f1.std() * 2)
    
    print("Estimate cross validation F1_macro: %0.2f (+/- %0.2f)" % (cross_validation_score_f1.mean(), cross_validation_score_f1.std() * 2))
    print("F1_macro Cross validation scores: " + str(cross_validation_score_f1))
    return cross_validation_score_f1,estimated_f1

In [96]:
cross_validation_score_f1_base_xgb_r,estimated_f1_base_xgb_r = return_x_val_f1_macro(XGB,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Estimate cross validation F1_macro: 0.97 (+/- 0.01)
F1_macro Cross validation scores: [0.96266002 0.97611001 0.97509029 0.97221646 0.96438188]


In [97]:
cross_validation_score_f1_base_gbc_r,estimated_f1_base_gbc_r = return_x_val_f1_macro(GBC,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Estimate cross validation F1_macro: 0.91 (+/- 0.01)
F1_macro Cross validation scores: [0.90580875 0.90729506 0.91218463 0.90459094 0.89731111]


In [98]:
cross_validation_score_f1_base_rfc_r,estimated_f1_base_rfc_r = return_x_val_f1_macro(RFC,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was pa

Estimate cross validation F1_macro: 0.99 (+/- 0.00)
F1_macro Cross validation scores: [0.98662963 0.98982552 0.99117314 0.98859276 0.98440595]


In [99]:
cross_validation_score_f1_base_svc_r,estimated_f1_base_svc_r = return_x_val_f1_macro(svc,X_train_resampled2,y_train_resampled2)

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate cross validation F1_macro: 0.94 (+/- 0.01)
F1_macro Cross validation scores: [0.93599725 0.94760576 0.94818757 0.94474342 0.93725469]


In [101]:
results_x_val_r={'Classifier':["XGB Classifier","Gradient Boosting Classifier","Random Forest Classifier","SVC"],
'Cross validation accuracy with resampled data':[estimated_accuracy_base_xgb_r,estimated_accuracy_base_gbc_r,estimated_accuracy_base_rfc_r,estimated_accuracy_base_svc_r],
'Cross validation F1_macro with resampled data':[estimated_f1_base_xgb_r,estimated_f1_base_gbc_r,estimated_f1_base_rfc_r,estimated_f1_base_svc_r]}

In [102]:
score_report_df_x_val_base_r =pd.DataFrame(data=results_x_val_r,columns=['Classifier','Cross validation accuracy with resampled data','Cross validation F1_macro with resampled data'])

In [104]:
print("Base models 5folds cross validation (Resampled Train_Test Split 80:20)")
score_report_df_x_val_base_r

Base models 5folds cross validation (Resampled Train_Test Split 80:20)


,Classifier,Cross validation accuracy with resampled data,Cross validation F1_macro with resampled data
0,XGB Classifier,"(0.9705582028590879, 0.011450220932560842)","(0.9700917345566253, 0.011083407288021587)"
1,Gradient Boosting Classifier,"(0.9061266167460857, 0.009556634799652016)","(0.9054380985831475, 0.009627753752311638)"
2,Random Forest Classifier,"(0.9882062627637849, 0.00494248442749251)","(0.988125398923953, 0.004772351814089159)"
3,SVC,"(0.9433287950987065, 0.01090345696278827)","(0.9427577394051878, 0.010311817873475565)"


In [111]:
score_report_df_base_r

,Classifier,Resampling Accuracy,Resampling F1_macro
0,XGB Classifier,0.831650,0.757909
1,Gradient Boosting Classifier,0.780808,0.703733
2,Random Forest Classifier,0.828199,0.753421
3,SVC,0.810774,0.734234


In [112]:
# "Base models Train_Test Split 80:20 validation"

score_report_df

,Classifier,Accuracy,F1_macro
0,XGB Classifier,0.870455,0.804928
1,Gradient Boosting Classifier,0.847475,0.776302
2,Random Forest Classifier,0.876263,0.814413
3,SVC,0.837710,0.696662


In [113]:
score_report_df_x_val_base

,Classifier,Cross validation accuracy,Cross validation F1_macro
0,XGB Classifier,"(0.8695286195286196, 0.008904275785594567)","(0.8062406411928424, 0.01316810135582849)"
1,Gradient Boosting Classifier,"(0.8470328282828283, 0.0076123787519490884)","(0.7818493446057356, 0.01889367499062976)"
2,Random Forest Classifier,"(0.8719065656565655, 0.003888488034495354)","(0.8087621418773745, 0.009972924824283975)"
3,SVC,"(0.8361742424242424, 0.007852457152775046)","(0.6920517702741111, 0.01654214233144192)"


# TO DO
https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html


HERE ARE RIGHT TO-DO STEPS
1. oversample the minority class
2. train the classifier on the training folds
3. validate the classifier on the remaining fold




In [44]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
from sklearn.multiclass import OneVsRestClassifier

In [45]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [46]:
from pprint import pprint

# !!!!!for MULTI LABEL, set parameters!!!!!
# objective="multi:softmax"
# num_class=3 (func, non func, need repair) xgb_ovr_clf.estimator.get_params().keys() does NOT SHOW the param! 

# Use OneVsRestClassifier or  OneVsOneClassifier from sklearn for multi label



In [ ]:
xgb_ovr_clf=OneVsRestClassifier(xgboost.XGBClassifier(objective="multi:softmax",num_class=3))

In [11]:
from pprint import pprint
pprint(xgb_ovr_clf.get_params())

{'estimator': XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, objective='multi:softmax',
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=False, verbosity=None),
 'estimator__base_score': None,
 'estimator__booster': None,
 'estimator__colsample_bylevel': None,
 'estimator__colsample_bynode': None,
 'estimator__colsample_bytree': None,
 'estimator__gamma': None,
 'estimator__gpu_id': None,
 'estimator__importance_type': 'gain',
 'estimator__interaction_con

In [12]:
# another way to print estimators

xgb_ovr_clf.estimator.get_params().keys()

dict_keys(['objective', 'base_score', 'booster', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'gamma', 'gpu_id', 'importance_type', 'interaction_constraints', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'monotone_constraints', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'subsample', 'tree_method', 'validate_parameters', 'verbosity', 'num_class'])

In [13]:
random_grid = {#'estimator__n_estimators': [100,200,300],
               'estimator__max_depth': [6,10,16],
               #'estimator__validate_parameters': [True],
               'estimator__min_child_weight': [1,3],
               'estimator__gamma':[ 0.0, 0.1],
               'estimator__learning_rate':[0.2,0.3],
               'estimator__colsample_bytree':[0.3,0.5]}



In [14]:
XGB_random=RandomizedSearchCV(xgb_ovr_clf,param_distributions=random_grid ,
                              n_iter=5,scoring='roc_auc',n_jobs=-1,cv=2,verbose=10)

In [15]:
from datetime import datetime
# timing starts from this point for "start_time" variable
start_time = timer(None)

XGB_random.fit(X,y)
timer(start_time) # timing ends here for "start_time" variable

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 12.9min remaining: 12.9min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed: 15.0min remaining:  6.4min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 15.9min finished



 Time taken: 0 hours 19 minutes and 31.99 seconds.


In [16]:
print(XGB_random.best_params_)

{'estimator__min_child_weight': 1, 'estimator__max_depth': 16, 'estimator__learning_rate': 0.3, 'estimator__gamma': 0.1, 'estimator__colsample_bytree': 0.5}


In [17]:
print(XGB_random.best_estimator_)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5, gamma=0.1,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.3,
                                            max_delta_step=None, max_depth=16,
                                            min_child_weight=1, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_class=3, num_parallel_tree=None,
                                            objective='multi:softmax',
                                         

In [18]:
print(XGB_random.best_score_)

0.845713669339405


In [34]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
from sklearn.multiclass import OneVsRestClassifier

In [35]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [10]:
xgb_ovr_clf=OneVsRestClassifier(xgboost.XGBClassifier(objective="multi:softmax",num_class=3))

In [11]:
from pprint import pprint
pprint(xgb_ovr_clf.get_params())

{'estimator': XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, objective='multi:softmax',
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=False, verbosity=None),
 'estimator__base_score': None,
 'estimator__booster': None,
 'estimator__colsample_bylevel': None,
 'estimator__colsample_bynode': None,
 'estimator__colsample_bytree': None,
 'estimator__gamma': None,
 'estimator__gpu_id': None,
 'estimator__importance_type': 'gain',
 'estimator__interaction_con

In [13]:
random_grid2 = {'estimator__n_estimators': [100,200,300],
                'estimator__booster':['gbtree'],
               'estimator__max_depth': [16],
               'estimator__validate_parameters': [True],
               'estimator__min_child_weight': [1],
               'estimator__gamma':[0.1],
               'estimator__learning_rate':[0.3],
               'estimator__colsample_bytree':[0.5]}



In [14]:
XGB_random=RandomizedSearchCV(xgb_ovr_clf,param_distributions=random_grid2 ,
                              n_iter=5,scoring='roc_auc',n_jobs=-1,cv=2,verbose=10)

In [15]:
from datetime import datetime
# timing starts from this point for "start_time" variable
start_time = timer(None)

XGB_random.fit(X,y)
timer(start_time) # timing ends here for "start_time" variable

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=5. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  7.5min remaining: 14.9min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 14.9min remaining: 14.9min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed: 14.9min remaining:  7.4min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 23.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 23.1min finished


[10:55:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:58:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:02:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not b

In [16]:
print(XGB_random.best_params_)

{'estimator__validate_parameters': True, 'estimator__n_estimators': 300, 'estimator__min_child_weight': 1, 'estimator__max_depth': 16, 'estimator__learning_rate': 0.3, 'estimator__gamma': 0.1, 'estimator__colsample_bytree': 0.5, 'estimator__booster': 'gbtree'}


In [17]:
print(XGB_random.best_estimator_)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5, gamma=0.1,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.3,
                                            max_delta_step=None, max_depth=16,
                                            min_child_weight=1, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=300, n_jobs=None,
                                            num_class=3, num_parallel_tree=None,
                                            objective='multi:softmax',
                                     

In [18]:
print(XGB_random.best_score_)

0.8462567756753672


In [ ]:
# f1macro

In [22]:
xgb_ovr_clf=OneVsRestClassifier(xgboost.XGBClassifier(objective="multi:softmax",num_class=3))


In [23]:
random_grid2 = {'estimator__n_estimators': [300,400,500],
                'estimator__booster':['gbtree'],
               'estimator__max_depth': [10],
               'estimator__validate_parameters': [True],
               'estimator__min_child_weight': [1],
               'estimator__gamma':[0.1],
               'estimator__learning_rate':[0.3],
               'estimator__colsample_bytree':[0.5]}


In [24]:
XGB_random=RandomizedSearchCV(xgb_ovr_clf,param_distributions=random_grid2 ,
                              n_iter=5,scoring='f1_macro',n_jobs=-1,cv=2,verbose=10)

In [25]:
from datetime import datetime
# timing starts from this point for "start_time" variable
start_time = timer(None)

XGB_random.fit(X,y)
timer(start_time) # timing ends here for "start_time" variable

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=5. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed: 15.4min remaining: 30.8min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 20.1min remaining: 20.1min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed: 20.1min remaining: 10.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 34.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 34.6min finished


[12:26:43] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:31:00] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:35:07] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not b

In [26]:
print(XGB_random.best_params_)

{'estimator__validate_parameters': True, 'estimator__n_estimators': 500, 'estimator__min_child_weight': 1, 'estimator__max_depth': 10, 'estimator__learning_rate': 0.3, 'estimator__gamma': 0.1, 'estimator__colsample_bytree': 0.5, 'estimator__booster': 'gbtree'}


In [27]:
print(XGB_random.best_estimator_)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5, gamma=0.1,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.3,
                                            max_delta_step=None, max_depth=10,
                                            min_child_weight=1, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=500, n_jobs=None,
                                            num_class=3, num_parallel_tree=None,
                                            objective='multi:softmax',
                                     

In [28]:
print(XGB_random.best_score_)

0.8052778418038147


# Instead of X and y, use resampled train X and y 
# scoring = f1_macro

In [36]:
xgb_ovr_clf=OneVsRestClassifier(xgboost.XGBClassifier(objective="multi:softmax",num_class=3))


In [37]:
random_grid3 = {'estimator__n_estimators': [100,300],
                'estimator__booster':['gbtree'],
               'estimator__max_depth': [10],
               'estimator__validate_parameters': [True],
               'estimator__min_child_weight': [1],
               'estimator__gamma':[0.1],
               'estimator__learning_rate':[0.3],
               'estimator__colsample_bytree':[0.1]}

In [38]:
XGB_random=RandomizedSearchCV(xgb_ovr_clf,param_distributions=random_grid3 ,
                              n_iter=5,scoring='f1_macro',n_jobs=-1,cv=2,verbose=10)

In [39]:
from datetime import datetime
# timing starts from this point for "start_time" variable
start_time = timer(None)

XGB_random.fit(X_train_resampled,y_train_resampled)
timer(start_time) # timing ends here for "start_time" variable

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:  1.6min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  3.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  3.7min finished


[10:34:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:35:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:35:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not b

In [40]:
print(XGB_random.best_params_)

{'estimator__validate_parameters': True, 'estimator__n_estimators': 100, 'estimator__min_child_weight': 1, 'estimator__max_depth': 10, 'estimator__learning_rate': 0.3, 'estimator__gamma': 0.1, 'estimator__colsample_bytree': 0.1, 'estimator__booster': 'gbtree'}


In [41]:
print(XGB_random.best_estimator_)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.1, gamma=0.1,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.3,
                                            max_delta_step=None, max_depth=10,
                                            min_child_weight=1, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_class=3, num_parallel_tree=None,
                                            objective='multi:softmax',
                                     

In [42]:
print(XGB_random.best_score_)

0.23887809255388498


# scoring default = none
# Instead of X and y, use resampled train X and y 

In [44]:
xgb_ovr_clf=OneVsRestClassifier(xgboost.XGBClassifier(objective="multi:softmax",num_class=3))

In [45]:
random_grid3 = {'estimator__n_estimators': [100,300],
                'estimator__booster':['gbtree'],
               'estimator__max_depth': [10],
               'estimator__validate_parameters': [True],
               'estimator__min_child_weight': [1],
               'estimator__gamma':[0.1],
               'estimator__learning_rate':[0.3],
               'estimator__colsample_bytree':[0.1]}

In [46]:
XGB_random=RandomizedSearchCV(xgb_ovr_clf,param_distributions=random_grid3 ,
                              n_iter=5,n_jobs=-1,cv=2,verbose=10)

In [47]:
from datetime import datetime
# timing starts from this point for "start_time" variable
start_time = timer(None)

XGB_random.fit(X_train_resampled,y_train_resampled)
timer(start_time) # timing ends here for "start_time" variablev

C:\Users\yuka\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:  1.7min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  3.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  3.4min finished


[10:42:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:43:42] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:44:50] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not b

In [48]:
print(XGB_random.best_params_)

{'estimator__validate_parameters': True, 'estimator__n_estimators': 300, 'estimator__min_child_weight': 1, 'estimator__max_depth': 10, 'estimator__learning_rate': 0.3, 'estimator__gamma': 0.1, 'estimator__colsample_bytree': 0.1, 'estimator__booster': 'gbtree'}


In [49]:
print(XGB_random.best_estimator_)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.1, gamma=0.1,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.3,
                                            max_delta_step=None, max_depth=10,
                                            min_child_weight=1, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=300, n_jobs=None,
                                            num_class=3, num_parallel_tree=None,
                                            objective='multi:softmax',
                                     

In [50]:
print(XGB_random.best_score_)

0.36946902654867253


# DO NOT WORK!!!!! TO FIX!!!! 
# resampling train set
# scoring = ‘roc_auc_ovr’, 'roc_auc_ovo', 'roc_auc'


In [51]:
xgb_ovr_clf=OneVsRestClassifier(xgboost.XGBClassifier(objective="multi:softmax",num_class=3))

In [52]:
random_grid3 = {'estimator__n_estimators': [100,300],
                'estimator__booster':['gbtree'],
               'estimator__max_depth': [10],
               'estimator__validate_parameters': [True],
               'estimator__min_child_weight': [1],
               'estimator__gamma':[0.1],
               'estimator__learning_rate':[0.3],
               'estimator__colsample_bytree':[0.1]}

In [58]:
XGB_random=RandomizedSearchCV(xgb_ovr_clf,param_distributions=random_grid3 ,
                              n_iter=5, scoring ='roc_auc_ovo', n_jobs=-1,cv=2,verbose=10)

In [59]:
from datetime import datetime
# timing starts from this point for "start_time" variable
start_time = timer(None)

XGB_random.fit(X_train_resampled,y_train_resampled)
timer(start_time) # timing ends here for "start_time" variablev

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:  1.5min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.5min remaining:    0.0s


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

# resampled train
# scoring = accuracy


In [62]:
xgb_ovr_clf=OneVsRestClassifier(xgboost.XGBClassifier(objective="multi:softmax",num_class=3))

In [63]:
random_grid3 = {'estimator__n_estimators': [100,300],
                'estimator__booster':['gbtree'],
               'estimator__max_depth': [10],
               'estimator__validate_parameters': [True],
               'estimator__min_child_weight': [1],
               'estimator__gamma':[0.1],
               'estimator__learning_rate':[0.3],
               'estimator__colsample_bytree':[0.1]}

In [64]:
XGB_random=RandomizedSearchCV(xgb_ovr_clf,param_distributions=random_grid3 ,
                              n_iter=5, scoring ='accuracy', n_jobs=-1,cv=2,verbose=10)

In [65]:
from datetime import datetime
# timing starts from this point for "start_time" variable
start_time = timer(None)

XGB_random.fit(X_train_resampled,y_train_resampled)
timer(start_time) # timing ends here for "start_time" variablev

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:  1.7min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  3.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  3.7min finished


[11:08:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:09:54] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:11:04] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { importance_type, missing, n_estimators } might not b

In [66]:
print(XGB_random.best_params_)

{'estimator__validate_parameters': True, 'estimator__n_estimators': 300, 'estimator__min_child_weight': 1, 'estimator__max_depth': 10, 'estimator__learning_rate': 0.3, 'estimator__gamma': 0.1, 'estimator__colsample_bytree': 0.1, 'estimator__booster': 'gbtree'}


In [67]:
print(XGB_random.best_estimator_)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.1, gamma=0.1,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.3,
                                            max_delta_step=None, max_depth=10,
                                            min_child_weight=1, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=300, n_jobs=None,
                                            num_class=3, num_parallel_tree=None,
                                            objective='multi:softmax',
                                     

In [68]:
print(XGB_random.best_score_)

0.36946902654867253


# TO DO 

* set xgb params  'booster':'gbtree'. it returned ' not suported.....? but model was comuting this, so it should work! > WORKED! 
* set ramdom search param scoring='f1_macro'> worked! 
* set random search param return_train_score=True
* params set by code
* read https://www.kaggle.com/mommermi/resampling-gridsearch-lightgbm-magic

https://stackoverflow.com/questions/61453795/using-sklearns-randomizedsearchcv-with-smote-oversampling-only-on-training-fold

# Findings 
* X y set is ok
* X_train_resampled, y_train_resampled did not work even if the dataset type are the same as X and y
* **scoring = ‘roc_auc_ovr’, 'roc_auc_ovo', 'roc_auc' did not work**
* y_train_resampled has no issue since it has 1 or 0 in one of three labels. 



# Questions

* cross val with resampled X y train
* use resampled X and y train for grid/randomise search. 
* 

# Gridsearch
# cross val 
# save model 
# save df 
# submit compe and see ranking 



# Findings

* step 1 cross validation and get mean arrucacy with defaults hyperparameters
* step 2 gridsearh or random search and get ".best_score_"
* step 3 compare step1( mean of corss val socre ) and step2 's score (".best_score" becuase grid and random do CV as default) to see the improvemnt from step1 to step2 
https://towardsdatascience.com/hyperparameter-tuning-c5619e7e6624


# Grid VS Random Search

**Grid**
* a small number of hyperparameters.
* computationally expensive! 
* time comsuming 
* it is guaranteed to find the optimal combination of parameters supplied

**Random**
* "There is a tradeoff to decreased processing time, however. We aren’t guaranteed to find the optimal combination of hyperparameters" 
* less expensive
* change "n_iter" value by yourself. Selecting too low of a number will decrease our chance of finding the best combination. Selecting too large of a number will increase our processing time.
* Use when the number of parameters to consider is particularly high and the magnitudes of influence are imbalanced
https://towardsdatascience.com/machine-learning-gridsearchcv-randomizedsearchcv-d36b89231b10



In [13]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [14]:
XGB =XGBClassifier()
#XGB.fit(X_train_resampled,y_train_resampled)
#y_pred =XGB.predict(X_test)
#print(classification_report(y_test,y_pred))
#print(accuracy_score(y_test, y_pred))

# FIX THIS CODE! infinity loop!!!!!!! ?????????????? 

In [16]:
# Number of trees
#n_estimator = [int(x) for x in np.linspace(start = 100, stop =300, num = 3)]

# booster types
booster = ['gbtree']

# Maximum number of levels in tree
max_dep = [int(x) for x in np.linspace(4, 10, num = 3)]
#max_dep.append(None)

# When set to True, XGBoost will perform validation of input parameters to check whether a parameter is used or not.
#validate_parameter = [True]

# the minimum weight (or number of samples if all samples have a weight of 1) required in order to create a new node in the tree.
#Smaller weight, smaller samples. If too big, will result in overfitiing
min_child_wei = [int(x) for x in np.linspace(1, 3, num = 3)]
#min_child_wei.append(None)

# learning_rate (eta) range: [0,1], default=0.3,  prevents overfitting
#eta = [0.1,0.3,0.5]

# Method of selecting samples for training each tree
bootstp = [True]#,False]


In [17]:
random_grid = {#'n_estimators': n_estimator,
               'booster':booster,
               'max_depth': max_dep,
               #'validate_parameters': validate_parameter,
               'min_child_weight': min_child_wei,
               #'learning_rate':eta,
               'bootstrap': bootstp}


In [18]:
random_grid

{'booster': ['gbtree'],
 'max_depth': [4, 7, 10],
 'min_child_weight': [1, 2, 3],
 'bootstrap': [True]}

In [22]:
# Use the random grid to search for best hyperparameters

# Random search of parameters, using 10 fold cross validation "cv = 5"
# search across 100 different combinations, and use all available cores n_iter = 100
# evaluate accuracy scoring='f1'
# n_iter :when a big number is set, it will take long to compute! default is 10

In [20]:
XGB_random.best_params_

{'min_child_weight': 1,
 'max_depth': 10,
 'bootstrap': True,
 'booster': 'gbtree'}

In [21]:
XGB_random.best_score_

0.9799093667032123

# GridSearch 
# Create the parameter grid based on the results of random search 

In [23]:
from sklearn.model_selection import GridSearchCV

In [25]:
n_estimator = [100,200,300]

booster = ['gbtree']

max_dep = [9, 11,13]

min_child_wei = [1]

eta =  [0.1,0.3,0.5]

bootstp = [True]

validate_param = [True]


In [26]:
grids_params = {'n_estimators': n_estimator,
               'booster':booster,
               'max_depth': max_dep,
               'validate_parameters': validate_param,
               'min_child_weight': min_child_wei,
               'learning_rate':eta,
               'bootstrap': bootstp}

In [27]:
gridsearch = GridSearchCV(estimator = XGB, param_grid = grids_params, scoring='f1_macro',
                          cv = 5, n_jobs = -1, verbose = 2, return_train_score=True)

In [28]:
gridsearch.fit(X_train_resampled,y_train_resampled)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 106.7min


KeyboardInterrupt: 

In [ ]:
gridsearch.best_params_

In [ ]:
gridsearch.best_socre_

In [17]:
import time
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import metrics
from pprint import pprint

In [ ]:
# Cross validation and compare randomised and grid search results

In [24]:
# Random search parameters
xgb_ran = XGBClassifier(min_child_weight =1,max_depth=10,bootstrap=True,booster="gbtree",
                       n_estimators=200,
                       validate_parameters=True,
                       learning_rate=0.3)

In [25]:
pprint(xgb_ran.get_params())

{'base_score': None,
 'booster': 'gbtree',
 'bootstrap': True,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': 0.3,
 'max_delta_step': None,
 'max_depth': 10,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 200,
 'n_jobs': None,
 'num_parallel_tree': None,
 'objective': 'binary:logistic',
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': True,
 'verbosity': None}


In [26]:
#cross_val_score(xgb_ran, X, y)

[11:59:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:03:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:06:17] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, mi

array([0.87685185, 0.87727273, 0.8773569 , 0.87718855, 0.87483165])

In [27]:
cross_validation_score = cross_val_score(xgb_ran, X, y)
print("Estimate the Accuracy: %0.2f (+/- %0.2f)" % (cross_validation_score.mean(), cross_validation_score.std() * 2))

[12:16:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:19:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:23:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, mi

In [28]:
#cross_val_score(xgb_ran,X, y,cv =5,scoring='f1_macro')


[12:33:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:36:55] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:44:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, mi

array([0.81517193, 0.81913602, 0.81183915, 0.82508796, 0.80384096])

In [29]:
cross_validation_score_f1 = cross_val_score(xgb_ran, X, y,scoring='f1_macro')
print("Estimate F1-macro: %0.2f (+/- %0.2f)" % (cross_validation_score_f1.mean(), cross_validation_score_f1.std() * 2))

[12:54:05] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:57:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:00:43] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { bootstrap, importance_type, mi

In [ ]:
# grid search parameters

xgb_grid = XGBClassifier()

In [ ]:
cross_val_score(xgb_grid, X, y)

In [ ]:
cross_validation_score_grid = cross_val_score(xgb_grid, X, y)
print("Estimate the Accuracy: %0.2f (+/- %0.2f)" % (cross_validation_score_grid.mean(), cross_validation_score_grid.std() * 2))

In [ ]:
cross_val_score(xgb_grid,X, y,cv =5,scoring='f1_macro')

In [ ]:
cross_validation_score_f1_grid = cross_val_score(xgb_grid, X, y,scoring='f1_macro')
print("Estimate F1-macro: %0.2f (+/- %0.2f)" % (cross_validation_score_f1_grid.mean(), cross_validation_score_f1_grid.std() * 2))

In [29]:
params = {'min_child_weight': 1,
 'max_depth': 10,
 'bootstrap': True,
 'booster': 'gbtree'}

In [30]:
XGB =XGBClassifier(params)
XGB.fit(X_train_resampled,y_train_resampled)
y_pred =XGB.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.84      0.86      6457
           1       0.43      0.76      0.55       851
           2       0.88      0.84      0.86      4572

    accuracy                           0.83     11880
   macro avg       0.74      0.81      0.76     11880
weighted avg       0.86      0.83      0.84     11880

0.8316498316498316


In [32]:
XGB =XGBClassifier(params)
XGB.fit(X_train_resampled,y_train_resampled)
y_pred =XGB.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.84      0.86      6457
           1       0.43      0.76      0.55       851
           2       0.88      0.84      0.86      4572

    accuracy                           0.83     11880
   macro avg       0.74      0.81      0.76     11880
weighted avg       0.86      0.83      0.84     11880

0.8316498316498316


In [26]:

GBC = GradientBoostingClassifier()
GBC.fit(X_train_resampled,y_train_resampled)
y_pred =GBC.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.88      0.78      0.83      6457
           1       0.32      0.80      0.46       851
           2       0.88      0.78      0.83      4572

    accuracy                           0.78     11880
   macro avg       0.69      0.79      0.70     11880
weighted avg       0.84      0.78      0.80     11880

0.7808080808080808


In [27]:

RFC =RandomForestClassifier()
RFC.fit(X_train_resampled,y_train_resampled)
y_pred =RFC.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.90      0.82      0.86      6457
           1       0.42      0.76      0.54       851
           2       0.87      0.84      0.86      4572

    accuracy                           0.83     11880
   macro avg       0.73      0.81      0.75     11880
weighted avg       0.86      0.83      0.84     11880

0.8271043771043771


In [28]:
svc =SVC()
svc.fit(X_train_resampled,y_train_resampled)
y_pred =svc.predict(X_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      6457
           1       0.39      0.75      0.51       851
           2       0.87      0.81      0.84      4572

    accuracy                           0.81     11880
   macro avg       0.71      0.79      0.73     11880
weighted avg       0.84      0.81      0.82     11880

0.8107744107744108


In [44]:
results={'Classifier':["XGB Classifier","Gradient Boosting Classifier","Random Forest Classifier","SVC"],
'Accuracy':[0.8316,0.7808,0.8276,0.8107],
'F1_macro':[0.76,0.70,0.75,0.73]}


In [45]:
score_report_df =pd.DataFrame(data=results,columns=["Classifier","Accuracy","F1_macro"])
score_report_df 

,Classifier,Accuracy,F1_macro
0,XGB Classifier,0.8316,0.76
1,Gradient Boosting Classifier,0.7808,0.70
2,Random Forest Classifier,0.8276,0.75
3,SVC,0.8107,0.73


In [46]:
from sklearn.metrics import f1_score

In [47]:
def modeling(clf,X_train_resampled,y_train_resampled,X_test,y_test):
    clf.fit(X_train_resampled,y_train_resampled)
    y_pred =clf.predict(X_test)
    clf_report = classification_report(y_test,y_pred)
    clf_acc_score = accuracy_score(y_test, y_pred)
    f1score=f1_score(y_test, y_pred, average='macro')
    return  clf_report, clf_acc_score, f1score
    print(clf_report)


In [51]:
XGB =XGBClassifier()
XGB_clf_report, XGB_clf_acc_score, XGB_f1score = modeling(XGB,X_train_resampled,y_train_resampled,X_test)

GBC = GradientBoostingClassifier()
GBC_clf_report, GBC_clf_acc_score, GBC_f1score = modeling(GBC,X_train_resampled,y_train_resampled,X_test)


RFC =RandomForestClassifier()
RFC_clf_report, RFC_clf_acc_score, RFC_f1score = modeling(RFC,X_train_resampled,y_train_resampled,X_test)

svc =SVC()
svc_clf_report, svc_clf_acc_score, svc_f1score = modeling(svc,X_train_resampled,y_train_resampled,X_test)


In [52]:
print(svc_clf_report)

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      6457
           1       0.39      0.75      0.51       851
           2       0.87      0.81      0.84      4572

    accuracy                           0.81     11880
   macro avg       0.71      0.79      0.73     11880
weighted avg       0.84      0.81      0.82     11880



In [53]:
print(svc_clf_acc_score)

0.8107744107744108


In [54]:
print(svc_f1score)

0.7342336000625377


In [55]:
"{:.3f}".format(svc_clf_acc_score)

'0.811'

In [56]:
'%.3f' % svc_clf_acc_score

'0.811'

# F1 and accuracy df table

In [ ]:
results={'Classifier':["XGB Classifier","Gradient Boosting Classifier","Random Forest Classifier","SVC"],
'Accuracy':[XGB_clf_acc_score,GBC_clf_acc_score,RFC_clf_acc_score,svc_clf_acc_score],
'F1_macro':[XGB_f1score,GBC_f1score,RFC_f1score,svc_f1score]}

In [ ]:
score_report_df =pd.DataFrame(data=results,columns=["Classifier","Accuracy","F1_macro"])
print(score_report_df)